# MODELE DE CHATBOT

Modèle de chatbot simple ne s'appuyant que sur des couches entièrement connectées afin d'être éxécutable sur la STM32

Réalisé par Elisa Delhommé

Source: https://towardsdatascience.com/a-simple-chatbot-in-python-with-deep-learning-3e8669997758
| https://moncoachdata.com/blog/construire-un-chatbot-en-python/

Les versions suivantes sont nécessaires pour être compatibles avec la version de CubeAI utilisée ultérieurement (v 5.1.2).

In [ ]:
!pip install q tensorflow==2.0.0
!pip install q keras==2.3.1

     |████████████████████████████████| 86.3 MB 38 kB/s 
     |████████████████████████████████| 449 kB 56.5 MB/s 
     |████████████████████████████████| 3.8 MB 44.2 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=8e0eb22e946981345b8785c3bf67291fdc7f3311aa599ee3296a93c96d8e9203
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Success

L'absence de couches d'Embedding (non supportée par la version de CubeAI employée, ce qui empêche par ailleurs l'utilisation de couches récurrentes telles que les LSTM ou RNN) nous mène à créer un dictionnaire afin de prédire la réponse au message porté par l'utilisateur.

La création d'un dictionnaire a également l'avantage d'avoir la possibilité de l'enrichir avec l'application recherchée et ainsi d'obtenir des réponses davantages ciblées.

Ici, on peut par exemple prévoir le cas où l'utilisateur demanderait l'emplacement de la machine à café par exemple.

In [ ]:
# utilisation d'un dictionnaire pour représenter un fichier JSON d'intentions
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "La forme?", "yo", "Salut", "ça roule?"],
              "responses": ["Salut à toi!", "Hello", "Comment vas tu?", "Salutations!", "Enchanté"],
             },
             {"tag": "age",
              "patterns": ["Quel âge as-tu?", "C'est quand ton anniversaire?", "Quand es-tu né?"],
              "responses": ["J'ai 25 ans", "Je suis né en 1996", "Ma date d'anniversaire est le 3 juillet et je suis né en 1996", "03/07/1996"]
             },
             {"tag": "date",
              "patterns": ["Que fais-tu ce week-end?",
"Tu veux qu'on fasse un truc ensemble?", "Quels sont tes plans pour cette semaine"],
              "responses": ["Je suis libre toute la semaine", "Je n'ai rien de prévu", "Je ne suis pas occupé"]
             },
             {"tag": "name",
              "patterns": ["Quel est ton prénom?", "Comment tu t'appelles?", "Qui es-tu?"],
              "responses": ["Mon prénom est Miki", "Je suis Miki", "Miki"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "Salut", "see ya", "adios", "cya"],
              "responses": ["C'était sympa de te parler", "à plus tard", "On se reparle très vite!"]
             },
             {"tag": "object_location",
              "patterns": [ "Where is the coffee maker?", "Où est la cafetière?"],
              "responses": ["On the kitchen table","Sur la table de la cuisine"]
             }
]}

In [ ]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Préparation des données d'entraînement

In [ ]:
# initialisation de lemmatizer pour obtenir la racine des mots
lemmatizer = WordNetLemmatizer()
# création des listes
words = []
classes = []
doc_X = []
doc_y = []
# parcourir avec une boucle For toutes les intentions
# tokéniser chaque pattern et ajouter les tokens à la liste words, les patterns et
# le tag associé à l'intention sont ajoutés aux listes correspondantes
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # ajouter le tag aux classes s'il n'est pas déjà là 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatiser tous les mots du vocabulaire et les convertir en minuscule
# si les mots n'apparaissent pas dans la ponctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# trier le vocabulaire et les classes par ordre alphabétique et prendre le
# set pour s'assurer qu'il n'y a pas de doublons
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
print(words)

['adios', 'anniversaire', 'as-tu', 'bye', "c'est", 'cafetière', 'ce', 'cette', 'coffee', 'comment', 'cya', 'ensemble', 'es-tu', 'est', 'fais-tu', 'fasse', 'forme', 'hello', 'is', 'la', 'maker', 'né', 'où', 'plan', 'pour', 'prénom', "qu'on", 'quand', 'que', 'quel', 'quels', 'qui', 'roule', 'salut', 'see', 'semaine', 'sont', "t'appelles", 'te', 'the', 'ton', 'truc', 'tu', 'un', 'veux', 'week-end', 'where', 'ya', 'yo', 'âge', 'ça']


In [ ]:
len(words)

51

In [ ]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name', 'object_location']


In [ ]:
print(doc_X)

['Hello', 'La forme?', 'yo', 'Salut', 'ça roule?', 'Quel âge as-tu?', "C'est quand ton anniversaire?", 'Quand es-tu né?', 'Que fais-tu ce week-end?', "Tu veux qu'on fasse un truc ensemble?", 'Quels sont tes plans pour cette semaine', 'Quel est ton prénom?', "Comment tu t'appelles?", 'Qui es-tu?', 'bye', 'Salut', 'see ya', 'adios', 'cya', 'Where is the coffee maker?', 'Où est la cafetière?']


In [ ]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'object_location', 'object_location']


In [ ]:
# liste pour les données d'entraînement
training = []
out_empty = [0] * len(classes)
# création du modèle d'ensemble de mots
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # marque l'index de la classe à laquelle le pattern atguel est associé à
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # ajoute le one hot encoded BoW et les classes associées à la liste training
    training.append([bow, output_row])
# mélanger les données et les convertir en array
random.shuffle(training)
training = np.array(training, dtype=object)
# séparer les features et les labels target
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
# définition de quelques paramètres
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200

## Développement du modèle

Le modèle permet la prédiction de la classe à laquelle appartient le message entré par l'utilisateur. A partir de cette classe, il pourra ensuite choisir une réponse adaptée pour renvoyer à l'utilisateur.

In [ ]:
# modèle Deep Learning
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
#model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               6656      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 15,302
Trainable params: 15,302
Non-trainable params: 0
_________________________________________________________________
None


## Entraînement du modèle

In [ ]:
# entraînement du modèle
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Train on 21 samples
Epoch 1/200
21/21 [==============================] - 1s 34ms/sample - loss: 1.7998 - accuracy: 0.2857
Epoch 2/200
21/21 [==============================] - 0s 218us/sample - loss: 1.5705 - accuracy: 0.7143
Epoch 3/200
21/21 [==============================] - 0s 207us/sample - loss: 1.3714 - accuracy: 0.8571
Epoch 4/200
21/21 [==============================] - 0s 207us/sample - loss: 1.1563 - accuracy: 0.9524
Epoch 5/200
21/21 [==============================] - 0s 189us/sample - loss: 0.9320 - accuracy: 0.9524
Epoch 6/200
21/21 [==============================] - 0s 277us/sample - loss: 0.7053 - accuracy: 0.9524
Epoch 7/200
21/21 [==============================] - 0s 249us/sample - loss: 0.4953 - accuracy: 0.9524
Epoch 8/200
21/21 [==============================] - 0s 272us/sample - loss: 0.3223 - accuracy: 0.9524
Epoch 9/200
21/21 [==============================] - 0s 288us/sample - loss: 0.2014 - accuracy: 0.9524
Epoch 10/200
21/21 [==============================] - 

## Fonctions d'encodage et décodage du texte

Fonction permettant de séparer une chaîne de caractères en plusieurs sous chaînes pour chacun des mots et ponctuations

<font color="lightgreen">
Exemple:
clean_text("comment ça va?") = ['comment', 'ça', 'va', '?']


In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

Fonction permettant l'encodage d'une chaîne de caractère pour traitement par le réseau de neurones
Précisément : met des 1 aux indices où un mot de la liste de référence (liste words) est repéré

<font color="lightgreen">
Exemple:
bag_of_words("comment ça va?", words) = array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                               0, 0, 0, 0, 0, 0, 1])
Les mots "comment" et "ça" ont été reconnus dans l'exemple

In [ ]:
def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

Fonction permettant de prédire la classe (le tag dans le dictionnaire) du message envoyé par l'utilisateur

<font color="lightgreen">
Exemple: pred_class("comment ça va?", words, classes) = ['greeting']

In [ ]:
def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #renvoi un array de 6 avec 
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh] #choisit le max des probas si supérieur à 0.2
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

In [ ]:
bow = bag_of_words("comment ça va?", words)

In [ ]:
result = model.predict(np.array([bow]))[0]
result

array([1.9173105e-07, 6.0139308e-07, 4.3546224e-05, 9.4537294e-01,
       5.4582629e-02, 8.6992578e-08], dtype=float32)

In [ ]:
thresh = 0.2
y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
y_pred

[[3, 0.94537294]]

In [ ]:
y_pred.sort(key=lambda x: x[1], reverse=True)
y_pred

[[3, 0.94537294]]

In [ ]:
return_list = []
for r in y_pred:
  return_list.append(classes[r[0]])
print(return_list)

['greeting']


Fonction permettant d'obtenir une réponse au message de l'utilisateur en choisissant de façon aléatoire une réponse qui coîncide dans le dictionnaire.

<font color="lightgreen">
Exemple:
get_response(['greeting'], data) = 'Salutations!'

In [ ]:
def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

## Test du chatbot

In [ ]:
# lancement du chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

yo
Hello


KeyboardInterrupt: ignored

In [ ]:
model_file_name = 'chatbot'
model.save(model_file_name + '.h5')